# Welcome to conu demo! 

You need to install following packages to be able to run docker examples: `docker` and `conu`, for kubernetes and OpenShift examples I recommend to run local cluster using `minishift`.

In [58]:
from IPython.display import display, HTML, Pretty

# Docker example

In [2]:
from conu import DockerBackend

In [3]:
backend = DockerBackend()

13:25:19.293 backend.py        INFO   conu has initiated, welcome to the party!
13:25:19.629 __init__.py       INFO   docker environment info: 'Client: Docker Engine - Community\n Version:           18.09.1\n API version:       1.39\n Go version:        go1.10.6\n Git commit:        4c52b90\n Built:             Wed Jan  9 19:33:12 2019\n OS/Arch:           darwin/amd64\n Experimental:      false\n\nServer: Docker Engine - Community\n Engine:\n  Version:          18.09.1\n  API version:      1.39 (minimum version 1.12)\n  Go version:       go1.10.6\n  Git commit:       4c52b90\n  Built:            Wed Jan  9 19:41:49 2019\n  OS/Arch:          linux/amd64\n  Experimental:     true\n'


In [4]:
image = backend.ImageClass("docker.io/library/nginx")

In [5]:
image.inspect()

{'Id': 'sha256:06144b2878448774e55577ae7d66b5f43a87c2e44322b3884e4e6c70d070b262',
 'RepoTags': ['nginx:latest'],
 'RepoDigests': ['nginx@sha256:24a0c4b4a4c0eb97a1aabb8e29f18e917d05abfe1b7a7c07857230879ce7d3d3'],
 'Parent': '',
 'Comment': '',
 'Created': '2018-09-05T00:57:00.322491744Z',
 'Container': 'ea0a37cdc91d1228880c77a4660fd74a6ba6ae5d2de6c54e2ac025c66c11f257',
 'ContainerConfig': {'Hostname': 'ea0a37cdc91d',
  'Domainname': '',
  'User': '',
  'AttachStdin': False,
  'AttachStdout': False,
  'AttachStderr': False,
  'ExposedPorts': {'80/tcp': {}},
  'Tty': False,
  'OpenStdin': False,
  'StdinOnce': False,
  'Env': ['PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin',
   'NGINX_VERSION=1.15.3-1~stretch',
   'NJS_VERSION=1.15.3.0.2.3-1~stretch'],
  'Cmd': ['/bin/sh', '-c', '#(nop) ', 'CMD ["nginx" "-g" "daemon off;"]'],
  'ArgsEscaped': True,
  'Image': 'sha256:674565b18e1c8e0102c657d89059818ba8e0f476073f63078439951d4e0370b0',
  'Volumes': None,
  'WorkingDir': '

In [6]:
container = image.run_via_binary()

13:25:27.513 image.py          INFO   run container via binary in background


In [7]:
container.is_running()

True

In [8]:
container.get_IPv4s()

['172.17.0.3']

In [9]:
container.get_ports()

['80']

In [10]:
container.stop()

In [11]:
container.is_running()

False

### HTTP client

In [12]:
image = backend.ImageClass("registry.fedoraproject.org/fedora")

In [14]:
container = image.run_via_binary(command=["python3", "-m", "http.server", "--bind", "0.0.0.0 8000"], additional_opts= ["-p", "8000:8000"])

13:26:18.683 image.py          INFO   run container via binary in background


In [15]:
container.get_ports()

['8000']

In [16]:
container.get_IPv4s()

['172.17.0.2']

In [17]:
container.get_port_mappings()

{'8000/tcp': [{'HostIp': '0.0.0.0', 'HostPort': '8000'}]}

In [18]:
request = container.http_request(host="127.0.0.1", port="8000")

In [19]:
request.ok

True

In [20]:
display(HTML(request.content.decode("utf-8")))

In [21]:
request = container.http_request(path="/etc", host="127.0.0.1", port="8000")

In [22]:
request.ok

True

In [23]:
display(HTML(request.content.decode("utf-8")))

In [24]:
"passwd" in request.content.decode("utf-8")

True

# Kubernetes example

This example demonstrating how to run image created by `DockerBackend` as container inside Kubernetes pod.

In [25]:
from conu import K8sBackend

In [26]:
k8s_backend = K8sBackend()

13:26:52.893 backend.py        INFO   conu has initiated, welcome to the party!


In [27]:
docker_backend = DockerBackend()

13:26:54.670 backend.py        INFO   conu has initiated, welcome to the party!


In [28]:
image = docker_backend.ImageClass("openshift/hello-openshift")

In [33]:
pod = image.run_in_pod(namespace='myproject')

13:27:12.048 image.py          INFO   Starting Pod hello-openshift-rpitonak-3pd9-pod in namespace myproject


In [35]:
pod.is_ready()

13:27:16.410 pod.py            INFO   Pod: hello-openshift-rpitonak-3pd9-pod in namespace: myproject is ready!


True

In [36]:
pod.get_phase()

<PodPhase.RUNNING: 1>

In [37]:
pod.delete()

13:27:21.171 pod.py            INFO   Deleting Pod hello-openshift-rpitonak-3pd9-pod in namespace myproject


In [38]:
pod.get_phase()

<PodPhase.TERMINATING: 4>

## Deployment

In [39]:
from conu.utils import get_oc_api_token

In [40]:
from conu.backend.k8s.deployment import Deployment

In [41]:
api_key = get_oc_api_token()
k8s_backend = K8sBackend(api_key=api_key)

13:27:27.969 backend.py        INFO   conu has initiated, welcome to the party!


In [42]:
template = """
        apiVersion: apps/v1
        kind: Deployment
        metadata:
          name: hello-world
          labels:
            app: hello-world
        spec:
          replicas: 3
          selector:
            matchLabels:
              app: hello-world
          template:
            metadata:
              labels:
                app: hello-world
            spec:
              containers:
              - name: hello-openshift
                image: openshift/hello-openshift
        """

In [43]:
test_deployment = Deployment(namespace='myproject', from_template=template, create_in_cluster=True)

13:27:31.793 deployment.py     INFO   Creating Deployment hello-world in namespace: myproject


In [45]:
test_deployment.all_pods_ready()

13:27:40.274 deployment.py     INFO   All pods are ready for deployment hello-world in namespace: myproject


True

In [46]:
test_deployment.get_status()

{'available_replicas': 3,
 'collision_count': None,
 'conditions': [{'last_transition_time': datetime.datetime(2019, 1, 21, 12, 27, 36, tzinfo=tzutc()),
                 'last_update_time': datetime.datetime(2019, 1, 21, 12, 27, 36, tzinfo=tzutc()),
                 'message': 'Deployment has minimum availability.',
                 'reason': 'MinimumReplicasAvailable',
                 'status': 'True',
                 'type': 'Available'},
                {'last_transition_time': datetime.datetime(2019, 1, 21, 12, 27, 30, tzinfo=tzutc()),
                 'last_update_time': datetime.datetime(2019, 1, 21, 12, 27, 36, tzinfo=tzutc()),
                 'message': 'ReplicaSet "hello-world-5d579c544d" has '
                            'successfully progressed.',
                 'reason': 'NewReplicaSetAvailable',
                 'status': 'True',
                 'type': 'Progressing'}],
 'observed_generation': 1,
 'ready_replicas': 3,
 'replicas': 3,
 'unavailable_replicas': None,
 '

In [47]:
test_deployment.delete()

13:27:45.515 deployment.py     INFO   Deleting Deployment hello-world in namespace: myproject


# OpenShift example

In [48]:
from conu.utils import get_oc_api_token

In [49]:
from conu import OpenshiftBackend

In [50]:
api_key = get_oc_api_token()

In [51]:
api_key

'CiytySi-38riHAe_s1wzHyY627usBeqGXxou8E0f9Jc'

In [52]:
openshift_backend = OpenshiftBackend(api_key=api_key)

13:27:57.195 backend.py        INFO   conu has initiated, welcome to the party!


In [53]:
app_name = openshift_backend.deploy_image("centos/mariadb-102-centos7",
                                         oc_new_app_args=["--env", "MYSQL_ROOT_PASSWORD=test"],
                                         project='myproject')

13:28:00.123 backend.py        INFO   Importing image from: centos/mariadb-102-centos7, as: mariadb-102-centos7
13:28:02.480 backend.py        INFO   Creating new app in project myproject


In [55]:
openshift_backend.all_pods_are_ready(app_name)

13:28:13.355 pod.py            INFO   Pod: app-ciwuc-1-df7lj in namespace: myproject is ready!
13:28:13.356 backend.py        INFO   All pods are ready!


True

In [59]:
logs = openshift_backend.get_logs(app_name)
display(Pretty(logs))

In project My Project (myproject) on server https://192.168.99.100:8443

svc/app-ciwuc - 172.30.38.63:3306
  dc/app-ciwuc deploys istag/mariadb-102-centos7:latest 
    deployment #1 deployed 2 minutes ago - 1 pod


2 infos identified, use 'oc status -v' to see details.
=> sourcing 20-validate-variables.sh ...
=> sourcing 25-validate-replication-variables.sh ...
=> sourcing 30-base-config.sh ...
---> 12:28:06     Processing basic MySQL configuration files ...
=> sourcing 60-replication-config.sh ...
=> sourcing 70-s2i-config.sh ...
---> 12:28:06     Processing additional arbitrary  MySQL configuration provided by s2i ...
=> sourcing 40-paas.cnf ...
=> sourcing 50-my-tuning.cnf ...
---> 12:28:06     Initializing database ...
---> 12:28:06     Running mysql_install_db ...
2019-01-21 12:28:06 140463009114304 [Note] Using unique option prefix 'ignore-db-dir' is error-prone and can break in the future. Please use the full name 'ignore_db_dirs' instead.
2019-01-21 12:28:06 140463009114304 [No

In [60]:
openshift_backend.clean_project(app_name)

13:30:50.905 backend.py        INFO   Deleting all objects with label app=app-ciwuc
13:30:52.647 backend.py        INFO   pod "app-ciwuc-1-df7lj" deleted
13:30:52.650 backend.py        INFO   replicationcontroller "app-ciwuc-1" deleted
13:30:52.651 backend.py        INFO   service "app-ciwuc" deleted
13:30:52.652 backend.py        INFO   deploymentconfig.apps.openshift.io "app-ciwuc" deleted
13:30:52.654 backend.py        INFO   


## Creating application from source

In [62]:
status = openshift_backend.get_status()
display(Pretty(status))

In project My Project (myproject) on server https://192.168.99.100:8443

You have no services, deployment configs, or build configs.
Run 'oc new-app' to create an application.


In [63]:
app_name = openshift_backend.create_new_app_from_source("centos/python-36-centos7",
                                                       source="https://github.com/openshift/django-ex.git",
                                                       project="myproject")

13:31:21.947 backend.py        INFO   Importing image from: centos/python-36-centos7, as: python-36-centos7
13:31:23.321 backend.py        INFO   Creating new app in project myproject


In [65]:
status = openshift_backend.get_status()
display(Pretty(status))

In project My Project (myproject) on server https://192.168.99.100:8443

svc/app-tiyaw - 172.30.223.124:8080
  dc/app-tiyaw deploys istag/app-tiyaw:latest <-
    bc/app-tiyaw source builds https://github.com/openshift/django-ex.git on istag/python-36-centos7:latest 
      build #1 running for 22 seconds - ab765c5: Merge pull request #134 from liangxia/okd (Honza Horak <hhorak@redhat.com>)
    deployment #1 waiting on image or update


2 infos identified, use 'oc status -v' to see details.


In [66]:
openshift_backend.all_pods_are_ready(app_name)

13:32:08.906 pod.py            INFO   Pod: app-tiyaw-1-99rtv in namespace: myproject is ready!
13:32:08.907 backend.py        INFO   All pods are ready!


True

In [67]:
openshift_backend.clean_project(app_name)

13:32:14.408 backend.py        INFO   Deleting all objects with label app=app-tiyaw
13:32:16.704 backend.py        INFO   pod "app-tiyaw-1-99rtv" deleted
13:32:16.708 backend.py        INFO   replicationcontroller "app-tiyaw-1" deleted
13:32:16.711 backend.py        INFO   service "app-tiyaw" deleted
13:32:16.712 backend.py        INFO   deploymentconfig.apps.openshift.io "app-tiyaw" deleted
13:32:16.715 backend.py        INFO   buildconfig.build.openshift.io "app-tiyaw" deleted
13:32:16.716 backend.py        INFO   build.build.openshift.io "app-tiyaw-1" deleted
13:32:16.720 backend.py        INFO   imagestream.image.openshift.io "app-tiyaw" deleted
13:32:16.724 backend.py        INFO   


For more information see upstream GitHub repository: https://github.com/user-cont/conu. You can find there more examples for all supported backends.